# Installation

In [ ]:
!pip install ultralytics

In [ ]:
!pip install gdown
# !gdown https://drive.google.com/uc?id=10o2U-Cfy2ZhckIT7vRmn0Y_4_AuOSqz1 -O chess_yolov8.pt
!gdown https://drive.google.com/uc?id=1o_ETLh04PBttb2cSzA8zZHmsxHsLYU76 -O chess_yolov8.pt

# preparing dataset

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import glob
from IPython.display import Image, display
import csv

# video_path = ['2_Move_student','2_Move_rotate_student','4_Move_student','6_Move_student','8_Move_student']

# chessboard detection

In [ ]:
def find_intersection(rho1, theta1, rho2, theta2):
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    B = np.array([rho1, rho2])
    return np.linalg.solve(A, B)

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
def get_points(image):
   # img = cv2.imread(image)
    img = image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 255, 255, apertureSize=3)

    # Hand Check
    lines = cv2.HoughLines(edges, 1, np.pi/180, 100)
    lines_tp = [(r_theta[0][0], r_theta[0][1]) for r_theta in lines ]

    tilted = False  # True -> Assume that there is a hand
    for r, theta in lines_tp:
        if (theta > np.pi/2 - 0.1 and theta < np.pi/2 + 0.1): continue
        if (theta > np.pi - 0.1 or theta < 0.1): continue
        tilted = True
        break

    # Find Lines
    lines = cv2.HoughLines(edges, 1, np.pi/180, 150)
    lines_tp = [(r_theta[0][0], r_theta[0][1]) for r_theta in lines ]

    rho = lines[:, 0, 0].copy()
    theta = lines[:, 0, 1].copy()

    for i in range(len(lines)):
        if rho[i] < 0:
            rho[i] = -rho[i]
            if theta[i] > np.pi - 0.1:
                theta[i] = 0

    rho_norm = rho / np.max(rho)
    theta_norm = theta / np.pi
    features = np.column_stack((rho_norm, theta_norm))

    # Apply DBSCAN
    eps = 0.05  # Adjust for sensitivity
    min_samples = 1
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(features)

    labels_set = set()
    lines_final = []
    horizontal_lines = []
    vertical_lines = []
    for i in range(len(labels)):
        if (labels[i] in labels_set or labels[i] == -1): continue
        if (lines_tp[i] == (1.0, 0.0)): continue
        labels_set.add(labels[i])
        lines_final.append(lines_tp[i])

        r, theta = lines_tp[i]
        if (theta > np.pi/2 - 0.1 and theta < np.pi/2 + 0.1):
            horizontal_lines.append((r, theta))
        elif (theta > np.pi - 0.1 or theta < 0.1):
            vertical_lines.append((r, theta))

    horizontal_lines = sorted(horizontal_lines)  # , key=lambda x: abs(x[0]))
    vertical_lines = sorted(vertical_lines, key=lambda x: abs(x[0]))
    if (len(horizontal_lines) != 9 or len(vertical_lines) != 9): 
        tilted = True
    
    # Find Intersection Points
    # grid = []
    # for h_rho, h_theta in horizontal_lines:
    #     row = []
    #     for v_rho, v_theta in vertical_lines:
    #         intersection = find_intersection(v_rho, v_theta, h_rho, h_theta)
    #         row.append((intersection[0], intersection[1]))  # Append [x, y]
    #     grid.append(row)

    # cells = dict()
    # for i in range(len(grid)-1):  # Loop through rows
    #     for j in range(len(grid[0])-1):  # Loop through columns
    #         top_left = grid[i][j]
    #         top_right = grid[i][j+1]
    #         bottom_left = grid[i+1][j]
    #         bottom_right = grid[i+1][j+1]
    #         cells[chr(ord("h") - j) + f"{i+1}"] = [top_left, top_right, bottom_left, bottom_right]

    grid = []
    for h_rho, h_theta in horizontal_lines:
        row = []
        for v_rho, v_theta in vertical_lines:
            intersection = find_intersection(v_rho, v_theta, h_rho, h_theta)
            row.append((intersection[0], intersection[1]))  # Append [x, y]
            # cv2.circle(img, (int(intersection[0]), int(intersection[1])), radius=5, color=(255, 0, 0), thickness=5) 
        grid.append(row)
        
    cells = dict()
    # for i in range(8):
    #     for j in range(8):
    #         cells[chr(ord("h") - j) + f"{i+1}"] = [[-1, -1], [-1, -1]]
    for i in range(min(8, len(grid)-1)):  # Loop through rows
        for j in range(min(8, len(grid[0])-1)):  # Loop through columns
            top_left = grid[i][j]
            top_right = grid[i][j+1]
            bottom_left = grid[i+1][j]
            bottom_right = grid[i+1][j+1]
            cells[chr(ord("h") - j) + f"{i+1}"] = [top_left, bottom_right]
            
            # font = cv2.FONT_HERSHEY_SIMPLEX
            # cv2.putText(img, chr(ord("h") - j) + f"{i+1}", (int(top_left[0]), int(top_left[1])), font, 1.5, (0,0,255), 4, cv2.LINE_AA)
    
    plt.imshow(img[:,:,::-1])
    plt.show()
    
    return tilted, cells

# Chess Pieces Detection

In [ ]:
import cv2
from ultralytics import YOLO #if cannot run -> make sure that using GPU
import matplotlib.pyplot as plt

In [ ]:
!dir

In [ ]:
def get_chess_pieces_position(image, chess_grid):
  #chess_board = ตารางที่มีตัวหมากรุก        chess_grid = แค่ช่องๆ
  chess_board = chess_board = {f"{chr(col)}{row}": None for col in range(ord('a'), ord('h') + 1) for row in range(1, 9)}
  # class_names =  ['bishop', 'black-bishop', 'black-king', 'black-knight', 'black-pawn', 'black-queen', 'black-rook', 'white-bishop', 'white-king', 'white-knight', 'white-pawn', 'white-queen', 'white-rook']  # Replace with your actual class names
  class_names =  ['black-bishop', 'black-king', 'black-knight', 'black-pawn', 'black-queen', 'black-rook', 'white-bishop', 'white-king', 'white-knight', 'white-pawn', 'white-queen', 'white-rook']  # Replace with your actual class names
  # class_names =  ['Black-Bishop', 'Black-King', 'Black-Knight', 'Black-Pawn', 'Black-Queen', 'Black-Rook', 'White-Bishop', 'White-King', 'White-Knight', 'White-Pawn', 'White-Queen', 'White-Rook']  # Replace with your actual class names
  # class_names = ['Black-bishop', 'Black-king', 'Black-knight', 'Black-pawn', 'Black-queen', 'Black-rook', 'White-bishop', 'White-king', 'White-knight', 'White-pawn', 'White-queen', 'White-rook']
    
  model = YOLO("chess_yolov8.pt")  # Replace with the path to your .pt file
  results = model(image)

  boxes = results[0].boxes
  class_ids = boxes.cls.cpu().numpy()  # Class IDs
  scores = boxes.conf.cpu().numpy()  # Confidence scores
  xyxy = boxes.xyxy.cpu().numpy()  # Bounding box coordinates
  centroids = [((box[0] + box[2]) / 2, box[3]) for box in xyxy]

  # hasHand, chess_grid = get_points(image)
  # print("hasHand", hasHand)
  # print("chess_grid", chess_grid)


  for idx, centroid in enumerate(centroids):
        # print("Resault From Yolo Id:", idx, "  Centroids:", centroid)
        x, y = centroid
        # Iterate over each grid cell and check if the centroid is inside the grid cell
        for grid_key, pst in chess_grid.items():
            # print("testttttt")
            # print(pst)
            x_min = pst[0][0]
            y_min = pst[0][1]
            x_max = pst[1][0]
            y_max = pst[1][1]
            # ((x_min, y_min), (x_max, y_max))
            # Check if the centroid (x, y) lies within the grid's boundaries
            if x_min <= x <= x_max and y_min <= y <= y_max:
                # If so, update the chess board with the class name
                chess_board[grid_key] = class_names[int(class_ids[idx])]
                break
  # image_cv = cv2.imread(image)  # Replace with the path to your image

  # # Convert from BGR (OpenCV default) to RGB (matplotlib default)
  # image_rgb = cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)
  # # Plot the image with Matplotlib
  # plt.figure(figsize=(10, 10))
  # plt.imshow(image_rgb)
  # # Draw bounding boxes and labels
  # for i in range(len(boxes)):
  #     x1, y1, x2, y2 = xyxy[i]
  #     class_id = int(class_ids[i])
  #     score = scores[i]

  #     # Get class name for each piece
  #     class_name = class_names[class_id]

  #     # Draw the rectangle on the image
  #     cv2.rectangle(image_rgb, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

  #     # Add class name and score as label
  #     label = f"{class_name}: {score:.2f}"
  #     plt.text(x1, y1, label, color="white", fontsize=12, backgroundcolor="blue")
  # # Display the image with bounding boxes and labels
  # plt.axis("off")
  # plt.show()

  # plt.figure(figsize=(10, 10))

  # # Draw bounding boxes and centroids
  # for idx, centroid in enumerate(centroids):
  #       x, y = centroid
  #       class_label = class_names[int(class_ids[idx])]  # Get the class label for the piece
  #       plt.scatter(x, y, color='red', s=10)  # Plot centroids as red dots
  #       # plt.text(x, y, f"{class_label}", color="white", fontsize=12, ha='center', backgroundcolor='blue')  # Add class label text

  #   # Draw the chessboard grid
  # for grid_key, ((x_min, y_min), (x_max, y_max)) in chess_grid.items():
  #     plt.plot([x_min, x_max], [y_min, y_min], color="blue")  # Top border
  #     plt.plot([x_min, x_max], [y_max, y_max], color="blue")  # Bottom border
  #     plt.plot([x_min, x_min], [y_min, y_max], color="blue")  # Left border
  #     plt.plot([x_max, x_max], [y_min, y_max], color="blue")  # Right border
  #     plt.text((x_min + x_max) / 2, (y_min + y_max) / 2, grid_key, color='black', fontsize=10, ha='center', va='center')  # Grid label

  # # Show the plot with the grid and centroids
  # plt.axis("off")
  # plt.show()
  return chess_board


# get_chess_pieces_position("/content/4_Move_student/frame_1800.jpg", {})



In [ ]:
import cv2

def get_chess_pieces_sequence(video_path, rotate):
    frames = []
    chess_sequence = {}
    cap = cv2.VideoCapture(video_path)  # Open the video file
    frame_count = 0
    processed_frame_count = 0  # Counter for processed frames

    while True:
        ret, frame = cap.read()  # Read the next frame

        if not ret:
            break  # If no frame is returned, exit the loop

        frame_count += 1

        # Process every 360th frame
        if frame_count % 360 == 0 or frame_count == 1:
            print(f"Processing frame {frame_count}")

            print("Rotate:", rotate)
            if rotate:
                frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
            hasHand, chess_grid = get_points(frame)
            # print("hasHand", hasHand)
            # print("chess_grid", chess_grid)

            # plt.figure(figsize=(10, 6))
            # plt.title(f"Frame {frame_count}")
            # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
            # plt.axis('off')  # Hide the axes
            # plt.show()

            # Get chess pieces positions for this frame
            if not hasHand:
              print("processing...")
              chess_board = get_chess_pieces_position(frame, chess_grid)  # Assuming this function is define
              print("testbanana")
              print(chess_board)
              chess_board = get_chess_pieces_position(frame, chess_grid)  # Assuming this function is define
              chess_sequence[frame_count] = chess_board
              frames.append(frame_count)
              processed_frame_count += 1
            else :
              print("not process, continue to next frame")
              continue

            # Store the chess board positions for the current frame

    cap.release()  # Release the video capture object
    print(f"Processed {processed_frame_count} frames.")
    return frames, chess_sequence

# Example usage
# print("frames ::: ", frames)
# for key, chess in chess_sequence.items():
#     print(key)    # Prints the frame number (key)
#     print(chess)



In [ ]:
def compare_chess_sequences(sequence1, sequence2):
    print("Ming2")
    print(sequence1)
    print(sequence2)
    move_list = []
    # Iterate through all keys in sequence1
    for key in sequence1:

        if key in sequence2 and sequence1[key] != sequence2[key]:
            move = {}
            found = False

            if(sequence1[key] != None and sequence2[key] == None ):
                print("just move")
                move['piece'] = sequence1[key]
                move["from"] = key

                for to_key in sequence2:
                  # print("TO KEY VALUE:", to_key)
                  # print("TO KEY 1:", sequence1[to_key], " TO KEY 2:", sequence2[to_key])
                  if(sequence2[to_key] == move["piece"]):
                    if(sequence1[to_key] == None):
                      move['type'] = "move"
                      move['to'] = to_key
                      found = True
                    else:
                      if(sequence1[to_key] != sequence2[to_key]):
                        move['type'] = "eat"
                        move['eatenPiece'] = sequence1[to_key]
                        move['to'] = to_key
                        found = True

            if(found):
              move_list.append(move)

    # print(1)
    return move_list

In [ ]:
def convert_to_pgn(moves):
    if (len(moves) == 0):
        return "1. "
    pgn = []
    # check black move first
    is_black_first = moves[0]["piece"].startswith("black")
    
    if is_black_first:
        black_move = convert_move(moves[0])
        pgn.append(f"1... {black_move}")
        start_index = 1  # next white turn
        move_number = 2  # 2nd move
    else:
        start_index = 0  
        move_number = 1 

    for i in range(start_index, len(moves), 2):
        white_move = convert_move(moves[i]) if i < len(moves) else ""
        black_move = convert_move(moves[i + 1]) if i + 1 < len(moves) else ""
        pgn.append(f"{move_number}. {white_move} {black_move}")
        move_number += 1  
    
    return " ".join(pgn)

def convert_move(move):
    piece = move["piece"]
    start = move["from"]
    end = move["to"]
    move_str = None

    # Castle
    if piece == "Castle" and start == "e1":
        move_str = "O-O" if end == "g1" else "O-O-O"

    # Pawn moves
    elif piece == "Pawn":
        if "promotion" in move:
            move_str = f"{end}={move['promotion']}"
        elif "en_passant" in move:
            move_str = f"{start[0]}x{end}"  # e.p."
        elif start[0] == end[0]:
            move_str = f"{end}"
        else:
            move_str = f"{start[0]}x{end}"

    # Other pieces
    else:
        symbols = {"black-knight": "N", "black-bishop": "B", "black-rook": "R", "black-queen": "Q", "black-king": "K","white-knight":"N","white-bishop":"B","white-rook":"R","white-queen":"Q","white-king":"K", "white-pawn": "", "black-pawn": ""}
        if move.get("disambiguate", False):
            move_str = f"{symbols.get(piece, piece[0].upper())}{start[0]}{end}"
        else:
            move_str = f"{symbols.get(piece, piece[0].upper())}{end}"

    # Add check or checkmate
    if move.get("checkmate", False):
        move_str += "#"
    elif move.get("check", False):
        move_str += "+"

    return move_str

# use this Project

In [ ]:
def use_this_project_each_video(video_path, rotate):
    move = []
    frames, chess_sequence = get_chess_pieces_sequence(video_path, rotate)
    print("Num Frames:", frames)
    # print("Sequences:", chess_sequence)
    for i in range(len(frames)-1):
        # print("Frame:", i)
        # print(f"compare between {frames[i]} and {frames[i+1]}" )
        change = compare_chess_sequences(chess_sequence[frames[i]], chess_sequence[frames[i+1]])
        move += change
        print("Change:", change)
        # print("Change:", change)
    # print("----------------------------------------------")
    return move


In [ ]:
import csv

def use_this_project(video_path_list):
    video_moves = {}

    # Process each video
    for video_path in video_path_list:
        rotate = False
        if "rotate" in video_path:
            rotate = True
        print("Processing Video Path:", video_path)
        # video_moves[video_path] = use_this_project_each_video("/kaggle/input/cu-chess-detection/test_videos/" + video_path, rotate)
        video_moves[video_path] = use_this_project_each_video("/kaggle/input/cu-chess-detection/Chess Detection Competition/test_videos/" + video_path, rotate)
    
    # Create CSV
    with open('submission.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["row_id", "output"])  # Write headers

        # Write moves for each video
        for video_id, moves in video_moves.items():
            pgn = convert_to_pgn(moves)
            writer.writerow([video_id, pgn])

        if (len(video_path_list) < 6):
            writer.writerow(["(Bonus)Long_video_student.mp4", "1. "])   # just for debug

    print("Save success!!")
    print("video moves:", video_moves)



In [ ]:
import os 

# path = "/kaggle/input/cu-chess-detection/test_videos"
path = "/kaggle/input/cu-chess-detection/Chess Detection Competition/test_videos"
video_path_list = os.listdir(path) 
video_path_list

In [ ]:
# use_this_project([video_path_list[1]])
use_this_project(video_path_list)

In [ ]:
with open("submission.csv", "r") as file:
    print(file.read())